# Análisis de Calidad del Café con Machine Learning

El presente análisis tiene como objetivo explorar y visualizar tendencias ocultas en las evaluaciones sensoriales del café, con base en datos recolectados por catadores profesionales. Utilizaremos técnicas de análisis exploratorio, reducción de dimensionalidad y clustering no supervisado para descubrir patrones interesantes que puedan ser útiles para la industria cafetera.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Cargar datos
df = pd.read_csv('/content/data-1.csv')

# Mostrar las primeras filas
df.head()


## Exploración Inicial de los Datos

Observamos las variables disponibles, su tipo, la presencia de valores nulos y la distribución de algunas métricas clave relacionadas con la calidad del grano de café.

In [ ]:

# Información general
df.info()

# Estadísticas descriptivas
df.describe()

# Revisar valores nulos
df.isnull().sum()


In [ ]:

# Seleccionamos únicamente las variables sensoriales que nos interesan
features = ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 
            'Uniformity', 'Clean Cup', 'Sweetness']

data = df[features]

# Eliminamos filas con valores faltantes
data_clean = data.dropna()

# Escalamos los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_clean)


## Representación Bidimensional con PCA

Utilizamos Análisis de Componentes Principales (PCA) para visualizar los datos en un espacio 2D. Esto nos permitirá detectar visualmente posibles agrupaciones o patrones de calidad en el café.

In [ ]:

pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)

# Visualización
plt.figure(figsize=(8,6))
plt.scatter(data_pca[:,0], data_pca[:,1], alpha=0.6, c='teal')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Proyección PCA de Evaluaciones Sensoriales')
plt.grid(True)
plt.show()


## Agrupamiento con KMeans

A continuación aplicamos KMeans para detectar grupos naturales en los datos embebidos. Usamos el coeficiente de silueta para determinar un número óptimo de clusters.

In [ ]:

# Prueba varios valores de K y mide el score
silhouette_scores = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(data_pca)
    score = silhouette_score(data_pca, labels)
    silhouette_scores.append(score)

# Visualizamos el mejor número de clusters
plt.plot(range(2, 10), silhouette_scores, marker='o')
plt.title('Silhouette Score para diferentes K')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()

# Elegimos el mejor número de clusters (mayor score)
best_k = np.argmax(silhouette_scores) + 2
kmeans = KMeans(n_clusters=best_k, random_state=42)
labels = kmeans.fit_predict(data_pca)

# Visualizamos clusters
plt.figure(figsize=(8,6))
sns.scatterplot(x=data_pca[:,0], y=data_pca[:,1], hue=labels, palette='Set2')
plt.title(f'Clusters de Evaluaciones Sensoriales (k={best_k})')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()


## Estadísticas por Grupo

Veamos ahora qué caracteriza a cada grupo detectado por KMeans. Esto puede ayudar a entender distintos perfiles sensoriales de café evaluados.

In [ ]:

data_clustered = data_clean.copy()
data_clustered['Cluster'] = labels

# Media por cluster
cluster_means = data_clustered.groupby('Cluster').mean()
cluster_means.T



## Conclusiones

Se identificaron **clusters diferenciados** entre los perfiles sensoriales del café, basados en evaluaciones de Aroma, Sabor, Retrogusto, Acidez, Cuerpo, Balance, Uniformidad, Taza Limpia y Dulzura.

Cada grupo representa un **perfil sensorial característico**, lo cual puede servir para clasificar granos de café según su experiencia de cata. Por ejemplo, hay clusters con alto Aroma y Dulzura pero bajo Cuerpo, mientras otros se distinguen por un mayor Balance y Retrogusto.

Estos hallazgos pueden ser útiles para:
- Guiar decisiones de compra y mezcla de granos.
- Personalizar recomendaciones de café según preferencias de sabor.
- Identificar oportunidades de mejora en la calidad de producción.

Se recomienda continuar el análisis con otros metadatos (origen, altura, procesamiento) si están disponibles.
